In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pymc3 as pm

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Define the Bayesian k-NN model
def bayesian_knn(x_train, y_train, x_test, k):
    with pm.Model() as model:
        # Prior for the class labels
        class_labels = pm.Categorical('class_labels', np.unique(y_train), shape=len(x_test))

        # Prior for the test point's class probabilities
        class_probs = pm.Dirichlet('class_probs', a=np.ones(len(np.unique(y_train))), shape=len(np.unique(y_train)))

        # K-nearest neighbors
        dists, inds = NearestNeighbors(n_neighbors=k).fit(x_train).kneighbors(x_test)
        knn_labels = y_train[inds]

        # Likelihood for the test points
        likelihood = pm.Categorical('likelihood', p=class_probs[class_labels], observed=knn_labels)

        # Perform Maximum a Posteriori estimation
        map_estimate = pm.find_MAP()

    return map_estimate

# Set the value of k
k = 3

# Split the dataset into training and test sets
train_indices = np.random.choice(range(len(X)), size=int(0.8*len(X)), replace=False)
test_indices = np.setdiff1d(range(len(X)), train_indices)
X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]

# Run Bayesian k-NN with MAP estimation
map_estimate = bayesian_knn(X_train, y_train, X_test, k)

# Generate predictions for the test set
test_predictions = []
for i in range(len(X_test)):
    knn_labels = y_train[NearestNeighbors(n_neighbors=k).fit(X_train).kneighbors([X_test[i]])[1][0]]
    class_probs = map_estimate['class_probs']
    predicted_label = np.argmax(np.bincount(knn_labels, weights=class_probs))
    test_predictions.append(predicted_label)

# Compute evaluation metrics for Bayesian k-NN
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average=None)
test_recall = recall_score(y_test, test_predictions, average=None)
test_f1_score = f1_score(y_test, test_predictions, average=None)

print("Bayesian k-NN Results:")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1-score: {test_f1_score}")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Bayesian k-NN Results:
Test Accuracy: 0.9666666666666667
Test Precision: [1.         1.         0.88888889]
Test Recall: [1.         0.91666667 1.        ]
Test F1-score: [1.         0.95652174 0.94117647]
